In [138]:
import json
import itertools
import re
import os
import pandas as pd
import chardet
from urllib.parse import urlparse
import csv
from collections import Counter

In [64]:
with open("data/ios_signatures.json", encoding="utf-8") as fh:
    signatures = json.load(fh)

In [65]:
regexs = []
for signature in signatures:
    regexs.append(signature["regex"])

# taken from: https://github.com/Exodus-Privacy/exodus-core
compiled_tracker_signature = [
    re.compile(signature["regex"], flags=re.MULTILINE | re.UNICODE)
    for signature in signatures
]

In [66]:
# taken from: https://github.com/Exodus-Privacy/exodus-core
def detect_trackers(filename):
    with open(filename, "r") as f:
        class_list = f.readlines()

    args = [
        (compiled_tracker_signature[index], tracker, class_list)
        for (index, tracker) in enumerate(signatures)
    ]

    results = []

    def _detect_tracker(sig, tracker, class_list):
        for clazz in class_list:
            if sig.search(clazz):
                return tracker
        return None

    for res in itertools.starmap(_detect_tracker, args):
        if res:
            results.append(res)

    trackers = [t["id"] for t in results if t is not None]

    return trackers


folder = "ios_log/classes/"
files = [
    fi for fi in os.listdir(folder) if not fi.startswith(".")
]  # ignores hidden files in dir

from multiprocess import Pool
import tqdm

from pathlib import Path


def detect(filename):
    if not filename.startswith("."):
        results = {}
        results["app"] = Path(filename).stem[0:-8]
        results["trackers"] = detect_trackers(folder + filename)
        return results


pool = Pool(processes=4)
trackers = {}

for filename in tqdm.tqdm(files, total=len(files)):
    file_path = os.path.join(folder, filename)
    print(f"Processing file: {file_path}")
    result = detect(filename)
    print(result, "\n")
    trackers[result["app"]] = result["trackers"]

for result in tqdm.tqdm(pool.imap_unordered(detect, files), total=len(files)):
    trackers[result["app"]] = result["trackers"]

with open("data_processed/ios_trackers.json", "w", encoding="utf-8") as f:
    json.dump(trackers, f)

  0%|          | 0/25 [00:00<?, ?it/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


Processing file: ios_log/classes/com.apalonapps.wlf-classes.txt


  4%|▍         | 1/25 [00:00<00:20,  1.18it/s]

{'app': 'com.apalonapps.wlf', 'trackers': [1, 3, 4, 5, 6, 7, 12, 13, 16, 31, 32, 34, 35, 48, 50, 57, 59, 61, 62, 80, 84, 85, 86]} 

Processing file: ios_log/classes/deluxeware.Weather-classes.txt


  8%|▊         | 2/25 [00:01<00:16,  1.43it/s]

...

 96%|█████████▌| 24/25 [00:17<00:00,  1.59it/s]

{'app': 'Com.BeeApp.MapWeather', 'trackers': [2, 3, 4, 5, 6, 7, 13, 38, 39, 50, 84, 85, 86, 92, 93, 94, 95, 98, 99, 107]} 

Processing file: ios_log/classes/com.mtmapping.onxhunt-classes.txt


100%|██████████| 25/25 [00:17<00:00,  1.39it/s]


{'app': 'com.mtmapping.onxhunt', 'trackers': [1, 3, 4, 6, 7, 26, 50, 84, 85, 86, 89, 93]} 



100%|██████████| 25/25 [00:07<00:00,  3.27it/s]


In [67]:
# This block creates a JSON file out of the third party tracker domains from TRACKER-RADAR
#
# Ignore this block if you already have a JSON file with third party domains

def list_files_in_directory(directory_path):
    # List to store file names without extensions
    file_names_without_extension = []

    # List all files in the directory
    for file_name in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, file_name)):
            # Remove file extension and add to the list
            name_without_extension = os.path.splitext(file_name)[0]
            file_names_without_extension.append(name_without_extension)

    return file_names_without_extension


def write_list_to_json(data, json_file_path):
    with open(json_file_path, "w") as json_file:
        json.dump(data, json_file, indent=4)


# Directory path for tracking domains
directory_path = "./tracker-radar-main/domains/US/"

# JSON file path
json_file_path = "./data/Third-Party-Trackers.json"

# Get file names without extensions
file_names = list_files_in_directory(directory_path)

# Write to JSON file
write_list_to_json(file_names, json_file_path)

In [116]:
# This block creates a CSV file containing app names and the corresponding Third-Party Trackers
# they contacted and the number of times they contacted them

def read_domains_from_json(json_file_path):
    with open(json_file_path, "r") as file:
        return set(json.load(file))


def process_all_har_files(har_directory, json_file_path, output_csv_path):
    tracker_domains = read_domains_from_json(json_file_path)
    all_files_results = {}

    for filename in os.listdir(har_directory):
        if filename.endswith(".har"):
            har_file_path = os.path.join(har_directory, filename)
            domain_counts = Counter()

            with open(har_file_path, "r") as file:
                har_data = json.load(file)

            for entry in har_data["log"]["entries"]:
                full_url = entry["request"]["url"]
                parsed_url = urlparse(full_url)
                domain = parsed_url.netloc.split(".")[-2:]  # Get the SLD and TLD
                domain_counts[".".join(domain)] += 1

            # Filter domains against the tracker list and store the results
            filtered_counts = {
                domain: count
                for domain, count in domain_counts.items()
                if domain in tracker_domains
            }
            all_files_results[filename.replace(".har", "")] = filtered_counts

    # Write to CSV
    with open(output_csv_path, "w", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["App Name", "Domain Counts"])

        for file_name, domains in all_files_results.items():
            domain_list = [f"{domain} ({count})" for domain, count in domains.items()]
            csvwriter.writerow([file_name, ", ".join(domain_list)])


# Define paths
har_directory = "./ios_log/har/"
json_file_path = "./data/Third-Party-Trackers.json"
output_csv_path = "./data/TrackersWithCount.csv"

# Process all .har files and write to CSV
process_all_har_files(har_directory, json_file_path, output_csv_path)

In [171]:
df = pd.read_csv('data/appsIos.csv')

set(df["permissions"])

{"['AllowsLocalNetworking', 'BluetoothAlwaysUsage', 'BluetoothPeripheralUsage', 'CalendarsUsage', 'CameraUsage', 'FaceIDUsage', 'LocationAlwaysAndWhenInUseUsage', 'LocationAlwaysUsage', 'LocationWhenInUseUsage', 'MicrophoneUsage', 'MotionUsage', 'PhotoLibraryUsage', 'UserTrackingUsage']",
 "['AppleMusicUsage', 'BluetoothAlwaysUsage', 'BluetoothPeripheralUsage', 'CalendarsUsage', 'CameraUsage', 'ContactsUsage', 'FaceIDUsage', 'LocationAlwaysAndWhenInUseUsage', 'LocationAlwaysUsage', 'LocationUsage', 'LocationWhenInUseUsage', 'MicrophoneUsage', 'PhotoLibraryUsage', 'SiriUsage', 'UserTrackingUsage']",
 "['AppleMusicUsage', 'BluetoothAlwaysUsage', 'BluetoothPeripheralUsage', 'CameraUsage', 'ContactsUsage', 'FaceIDUsage', 'LocationAlwaysAndWhenInUseUsage', 'LocationAlwaysUsage', 'LocationWhenInUseUsage', 'PhotoLibraryUsage', 'UserTrackingUsage']",
 "['BluetoothAlwaysUsage', 'BluetoothPeripheralUsage', 'CalendarsUsage', 'CameraUsage', 'ContactsUsage', 'LocalNetworkUsage', 'LocationAlwaysAndW